In [ ]:
import rich
from sotopia.database.persistent_profile import AgentProfile, EnvironmentProfile, RelationshipProfile
from sotopia.database.logs import EpisodeLog
from collections import Counter 
from redis_om import Migrator 

## Relationship Profile

In [ ]:
res_pks = RelationshipProfile.all_pks()
res_pks = list(res_pks)
print(len(res_pks))
res = []
for pk in res_pks:
    print(pk)
    try:
        res.append(RelationshipProfile.get(pk=pk))
    except:
        print("error")
        pass
res_relationships = [r.relationship for r in res]
Counter(res_relationships)

## Agents Profile

In [ ]:
# obtain a specific agent
agents = AgentProfile.find(AgentProfile.first_name == "ss").all()
rich.print(agents)

In [ ]:
# find specific agnets
agents = AgentProfile.find(AgentProfile.gender=="Man", AgentProfile.age>30)
for agent in agents:
    rich.print(agent)

In [ ]:
# obtain all agents' basic info
agent_pks = AgentProfile.all_pks()
agent_pks = list(agent_pks)
print(len(agent_pks))
agents = []
for pk in agent_pks:
    try:
        agents.append(AgentProfile.get(pk=pk))
    except:
        print("error")
        pass
# output agents's basic info
for agent in agents:
    rich.print(agent)

In [ ]:
# Update agent's information
agents = AgentProfile.find(AgentProfile.first_name == 'Ava', AgentProfile.last_name == 'Martinez').all()[0]

In [ ]:
agents

In [ ]:
agents.update(secret = 'Keeps their bisexuality a secret from her conservative family')

## Environment Profile

In [ ]:
env_profile_id = "01H6S9W1BMGR7MFRPH0V55J2TD"
env = EnvironmentProfile.get(env_profile_id)
rich.print(env)

## Episode Log

In [ ]:
# tag: 6_initial_aug2
# Migrator().run()
Episodes = EpisodeLog.find(EpisodeLog.tag == "6_initial_aug14_full").all()

In [ ]:
# get a human readable version of the episode
agent_profiles, conversation = Episodes[1].render_for_humans()
for agent_profile in agent_profiles:
    rich.print(agent_profile)
for message in conversation:
    rich.print(message)

In [ ]:
len(set([Episode.environment for Episode in Episodes]))

In [ ]:
from collections import defaultdict
import pandas as pd

def get_avg_reward_for_models(episodes: list[EpisodeLog]) -> dict[str, dict[str, float]]:
    """Get the average reward for each model in the episodes.

    Args:
        episodes (list[EpisodeLog]): A list of episodes.

    Returns:
        dict[str, float]: A dictionary mapping model names to average rewards.
    """
    model_rewards = defaultdict(list)
    model_rewards_avg = {}
    for episode in episodes:
        for idx, model in enumerate(episode.models[1:]): # skip env
            if isinstance(episode.rewards[idx], tuple):
                model_rewards[model].append(episode.rewards[idx])
            else:
                #rich.print(episode.render_for_humans())
                print(episode.rewards)
                #print(episode.rewards[idx])

    for model in model_rewards:
        model_rewards[model] = [rewards[1] for rewards in model_rewards[model]] 
        model_rewards_avg[model] = pd.DataFrame.from_dict(model_rewards[model])
        model_rewards_avg[model] = model_rewards_avg[model].mean(axis=0).to_dict()
    return pd.DataFrame.from_dict(model_rewards_avg)

def get_avg_successRate_for_models(episodes: list[EpisodeLog]) -> dict[str, dict[str, float]]:
    """Get the average success rate for each model in the episodes.

    Args:
        episodes (list[EpisodeLog]): A list of episodes.

    Returns:
        dict[str, float]: A dictionary mapping model names to average success rates.
    """
    model_rewards = defaultdict(list)
    model_successRate_avg = {}
    for episode in episodes:
        for idx, model in enumerate(episode.models[1:]): # skip env
            if isinstance(episode.rewards[idx], tuple):
                model_rewards[model].append(episode.rewards[idx])
            else:
                #rich.print(episode.render_for_humans())
                print(episode.rewards, episode.messages[0])
                #print(episode.rewards[idx])

    for model in model_rewards:
        model_successRate_avg[model] = [rewards[1] for rewards in model_rewards[model]]
        model_successRate_avg[model] = pd.DataFrame.from_dict(model_successRate_avg[model])
    assert len(model_successRate_avg) == 2, "There should be two models"
    model_list = list(model_successRate_avg.keys())
    model_one_successRate = model_successRate_avg[model_list[0]] > model_successRate_avg[model_list[1]]
    model_two_successRate = model_successRate_avg[model_list[0]] < model_successRate_avg[model_list[1]]
    model_on_par_successRate = model_successRate_avg[model_list[0]] == model_successRate_avg[model_list[1]]
    return pd.DataFrame.from_dict({
        model_list[0]: model_one_successRate.mean(axis=0).to_dict(),
        model_list[1]: model_two_successRate.mean(axis=0).to_dict(),
        "on_par": model_on_par_successRate.mean(axis=0).to_dict(),
    })

In [ ]:
get_avg_reward_for_models(Episodes)

In [ ]:
a = get_avg_successRate_for_models(Episodes)
a